In [1]:
API_ID = "alek4h7jlreffeoe5tocxgnx2u"
CAREER_SALARY_TABLE      = f"careerSalary-{API_ID}-NONE"
CAREER_EDUCATION_TABLE   = f"careerEducation-{API_ID}-NONE"
CAREER_SKILLS_TABLE      = f"careerSkills-{API_ID}-NONE"
CAREER_DESCRIPTION_TABLE = f"careerDescription-{API_ID}-NONE"
SOC_CODES_TABLE          = f"socCodes-{API_ID}-NONE"

In [ ]:
import os
import csv
import time
import threading
from decimal import Decimal
from datetime import datetime, timezone
import boto3
from queue import Queue

# Config
CLEANED_DIR = "Datasets/dynamodb_ready_by_year"
REGION = "us-west-2"
BATCH_SIZE = 1000
NUM_THREADS = 4
REJECTED_FILE = "rejected_dynamodb_rows.csv"

# AWS setup
dynamodb = boto3.resource("dynamodb", region_name=REGION)
table = dynamodb.Table(CAREER_SALARY_TABLE)

queue = Queue()
rejected_rows = []
lock = threading.Lock()

def to_decimal(val):
    try:
        return Decimal(str(round(float(val), 2))) if val not in [None, '', 'null'] else None
    except:
        return None

def current_timestamp():
    return datetime.now(timezone.utc).isoformat()

def process_batch(batch, batch_number):
    start = time.time()
    try:
        with table.batch_writer(overwrite_by_pkeys=['occCode']) as writer:
            for item in batch:
                try:
                    writer.put_item(Item=item)
                except Exception as e:
                    with lock:
                        rejected_rows.append({**item, 'error': str(e)})
        print(f"✅ Batch {batch_number} inserted in {time.time() - start:.2f}s ({len(batch)} records)")
    except Exception as e:
        with lock:
            for item in batch:
                rejected_rows.append({**item, 'error': str(e)})
        print(f"❌ Batch {batch_number} failed with error: {str(e)}")

def worker():
    batch_number = 1
    while True:
        batch = queue.get()
        if batch is None:
            break
        process_batch(batch, batch_number)
        batch_number += 1
        queue.task_done()

def read_and_queue_csv(filepath):
    with open(filepath, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        batch = []
        for row in reader:
            try:
                now = current_timestamp()
                item = {
                    'occCode': str(row['occ_code']),
                    'salaryKey': str(row['salary_key']),
                    'aMedian': to_decimal(row.get('a_median')),
                    'mMedian': to_decimal(row.get('m_median')),
                    'mPct10': to_decimal(row.get('m_pct10')),
                    'mPct90': to_decimal(row.get('m_pct90')),
                    'createdAt': now,
                    'updatedAt': now,
                }
                # Remove None fields
                item = {k: v for k, v in item.items() if v is not None}
                batch.append(item)
                if len(batch) == BATCH_SIZE:
                    queue.put(batch)
                    batch = []
            except Exception as e:
                with lock:
                    rejected_rows.append({**row, 'error': str(e)})

        if batch:
            queue.put(batch)

def save_rejected_rows():
    if rejected_rows:
        keys = rejected_rows[0].keys()
        with open(REJECTED_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(rejected_rows)
        print(f"⚠️ {len(rejected_rows)} records rejected. Saved to {REJECTED_FILE}")

# Main
print(f"🚀 Starting upload from all CSVs in: {CLEANED_DIR}")
start = time.time()

threads = []
for _ in range(NUM_THREADS):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

for filename in os.listdir(CLEANED_DIR):
    if filename.endswith("_dynamodb_ready.csv"):
        filepath = os.path.join(CLEANED_DIR, filename)
        print(f"📂 Queuing data from {filename}")
        read_and_queue_csv(filepath)

queue.join()

# Stop workers
for _ in range(NUM_THREADS):
    queue.put(None)
for t in threads:
    t.join()

save_rejected_rows()
print(f"🎉 All batches uploaded in {time.time() - start:.2f} seconds.")


🚀 Starting upload from all CSVs in: Datasets/dynamodb_ready_by_year
📂 Queuing data from oes_research_2016_allsectors_dynamodb_ready.csv
✅ Batch 1 inserted in 4.06s (1000 records)
✅ Batch 1 inserted in 4.30s (1000 records)
✅ Batch 1 inserted in 4.40s (1000 records)
✅ Batch 1 inserted in 4.48s (1000 records)
✅ Batch 2 inserted in 3.35s (1000 records)
✅ Batch 2 inserted in 3.43s (1000 records)
✅ Batch 2 inserted in 3.65s (1000 records)
✅ Batch 2 inserted in 3.99s (1000 records)
✅ Batch 3 inserted in 3.31s (1000 records)
✅ Batch 3 inserted in 3.66s (1000 records)
✅ Batch 3 inserted in 3.19s (1000 records)
✅ Batch 3 inserted in 3.83s (1000 records)
📂 Queuing data from oes_research_2017_allsectors_dynamodb_ready.csv
✅ Batch 4 inserted in 2.68s (1000 records)
✅ Batch 4 inserted in 3.09s (1000 records)
✅ Batch 4 inserted in 2.91s (1000 records)
✅ Batch 4 inserted in 3.66s (1000 records)
✅ Batch 5 inserted in 3.29s (1000 records)
✅ Batch 5 inserted in 3.81s (1000 records)
✅ Batch 5 inserted in 

KeyboardInterrupt: 

✅ Batch 15 inserted in 1.62s (1000 records)


✅ Batch 14 inserted in 1.67s (1000 records)
✅ Batch 14 inserted in 1.68s (1000 records)
✅ Batch 14 inserted in 1.55s (1000 records)
✅ Batch 16 inserted in 1.75s (1000 records)
✅ Batch 15 inserted in 1.73s (1000 records)
✅ Batch 15 inserted in 1.88s (1000 records)
✅ Batch 15 inserted in 1.77s (1000 records)
✅ Batch 17 inserted in 1.68s (1000 records)
✅ Batch 16 inserted in 1.65s (1000 records)
✅ Batch 16 inserted in 1.72s (1000 records)
✅ Batch 16 inserted in 1.76s (1000 records)
✅ Batch 18 inserted in 1.55s (1000 records)
✅ Batch 17 inserted in 1.62s (1000 records)
✅ Batch 17 inserted in 1.54s (1000 records)
✅ Batch 17 inserted in 1.64s (1000 records)


In [ ]:
#insert education data
import os
import csv
import time
import threading
from collections import defaultdict
from datetime import datetime, timezone
import boto3
from concurrent.futures import ThreadPoolExecutor, as_completed
from botocore.exceptions import ClientError

# Input CSV and table config
input_file = "Datasets/education_data.csv"
region = "us-west-2"

# Updated field mapping to match new schema (camelCase)
CODE_TO_FIELD = {
    'Less_than_hs': 'lessThanHs',
    'hs_or_eq': 'hsOrEq',
    'Associate_degree': 'associateDegree',
    'Bachelor_degree': 'bachelorDegree',
    'Master_degree': 'masterDegree',
    'Doctorate_degree': 'doctorateDegree',
    'No_requirement': 'noRequirement',
    'Professional_degree': 'professionalDegree',
}

# Thread-safe logging
print_lock = threading.Lock()
def log(msg):
    with print_lock:
        print(msg)

def current_timestamp():
    return datetime.now(timezone.utc).isoformat()

def read_and_reshape(input_file):
    """Reads the CSV and pivots to {occCode: {fields...}} for DynamoDB"""
    edu_data = defaultdict(dict)
    for row in csv.DictReader(open(input_file, newline='', encoding='utf-8')):
        occ = row['SOC']
        code = row['ESTIMATECODE']
        field = CODE_TO_FIELD.get(code)
        if not field:
            continue
        value = row['ESTIMATE']
        edu_data[occ].setdefault('occCode', occ)
        edu_data[occ][field] = value

    # Add timestamps to each record
    now = current_timestamp()
    for record in edu_data.values():
        record['createdAt'] = now
        record['updatedAt'] = now

    return list(edu_data.values())

def dynamodb_batch_write(table, items):
    with table.batch_writer(overwrite_by_pkeys=['occCode']) as batch:
        for item in items:
            batch.put_item(Item=item)

def batch_iterable(iterable, batch_size):
    batch = []
    for item in iterable:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

def insert_batches_singlethreaded(items, table):
    start = time.time()
    total = len(items)
    batch_num = 0
    for batch in batch_iterable(items, 25):
        batch_num += 1
        try:
            dynamodb_batch_write(table, batch)
            log(f"✅ Batch {batch_num} ({len(batch)}) inserted [{(batch_num-1)*25+1}-{batch_num*25}]")
        except ClientError as e:
            log(f"❌ Batch {batch_num} error: {e}")
    log(f"Total items inserted: {total}")
    return time.time() - start

def insert_batches_multithreaded(items, table, n_workers=4):
    start = time.time()
    batches = list(batch_iterable(items, 100))
    batch_num = 0
    def upload_batch(batch):
        nonlocal batch_num
        batch_num += 1
        try:
            dynamodb_batch_write(table, batch)
            log(f"✅ Batch {batch_num} ({len(batch)}) inserted")
        except ClientError as e:
            log(f"❌ Batch {batch_num} error: {e}")

    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(upload_batch, batch) for batch in batches]
        for f in as_completed(futures):
            pass
    log(f"Total items inserted: {len(items)}")
    return time.time() - start

if __name__ == "__main__":
    # 1. Read and reshape input
    log(f"🔄 Reading and pivoting CSV: {input_file}")
    items = read_and_reshape(input_file)
    log(f"📦 Total DynamoDB items to insert: {len(items)}")

    # 2. Setup DynamoDB
    session = boto3.Session(region_name=region)
    dynamodb = session.resource('dynamodb')
    table = dynamodb.Table(CAREER_EDUCATION_TABLE)

    # 3. Single-threaded insert
    log("🚀 Starting single-threaded DynamoDB insert...")
    t1 = insert_batches_singlethreaded(items, table)
    log(f"⏱️ Single-threaded insert time: {t1:.2f} sec")

    # 4. Multi-threaded insert
    log("🚀 Starting multi-threaded DynamoDB insert (4 threads)...")
    t2 = insert_batches_multithreaded(items, table, n_workers=4)
    log(f"⏱️ Multi-threaded insert time: {t2:.2f} sec")

    log("🎯 Script completed.")

🔄 Reading and pivoting CSV: Datasets/education_data.csv
📦 Total DynamoDB items to insert: 175
🚀 Starting single-threaded DynamoDB insert...
✅ Batch 1 (25) inserted [1-25]
✅ Batch 2 (25) inserted [26-50]
✅ Batch 3 (25) inserted [51-75]
✅ Batch 4 (25) inserted [76-100]
✅ Batch 5 (25) inserted [101-125]
✅ Batch 6 (25) inserted [126-150]
✅ Batch 7 (25) inserted [151-175]
Total items inserted: 175
⏱️ Single-threaded insert time: 1.05 sec
🚀 Starting multi-threaded DynamoDB insert (4 threads)...
✅ Batch 2 (100) inserted
✅ Batch 2 (75) inserted
Total items inserted: 175
⏱️ Multi-threaded insert time: 0.92 sec
🎯 Script completed.


In [ ]:
#insert career skills data
import os
import csv
import ast
import time
import threading
from datetime import datetime, timezone
import boto3
from queue import Queue

# CONFIGURATION
CSV_FILE = "Datasets/skills_data.csv"
REGION = "us-west-2"
BATCH_SIZE = 100
NUM_THREADS = 4
REJECTED_FILE = "rejected_career_skills.csv"

# AWS setup
dynamodb = boto3.resource("dynamodb", region_name=REGION)
table = dynamodb.Table(CAREER_SKILLS_TABLE)

queue = Queue()
rejected_rows = []
lock = threading.Lock()

def current_timestamp():
    return datetime.now(timezone.utc).isoformat()

def process_batch(batch, batch_number):
    start = time.time()
    try:
        with table.batch_writer(overwrite_by_pkeys=['occCode']) as writer:
            for item in batch:
                try:
                    writer.put_item(Item=item)
                except Exception as e:
                    with lock:
                        rejected_rows.append({**item, 'error': str(e)})
        print(f"✅ Batch {batch_number} inserted in {time.time() - start:.2f}s ({len(batch)} records)")
    except Exception as e:
        with lock:
            for item in batch:
                rejected_rows.append({**item, 'error': str(e)})
        print(f"❌ Batch {batch_number} failed with error: {str(e)}")

def worker():
    batch_number = 1
    while True:
        batch = queue.get()
        if batch is None:
            break
        process_batch(batch, batch_number)
        batch_number += 1
        queue.task_done()

def read_and_queue_csv(filepath):
    with open(filepath, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        batch = []
        for row_num, row in enumerate(reader, 2):
            try:
                occ_code = str(row['SOC_CODE'])
                skills_str = row['TYPICAL_SKILLS']
                skills = ast.literal_eval(skills_str)
                now = current_timestamp()
                item = {
                    'occCode': occ_code,
                    'skills': skills,
                    'createdAt': now,
                    'updatedAt': now
                }
                batch.append(item)
                if len(batch) == BATCH_SIZE:
                    queue.put(batch)
                    batch = []
                if row_num % 1000 == 0:
                    print(f"   ...Processed {row_num} rows so far")
            except Exception as e:
                with lock:
                    rejected_rows.append({**row, 'error': str(e)})
        if batch:
            queue.put(batch)

def save_rejected_rows():
    if rejected_rows:
        keys = rejected_rows[0].keys()
        with open(REJECTED_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(rejected_rows)
        print(f"⚠️ {len(rejected_rows)} records rejected. Saved to {REJECTED_FILE}")

# Main
print(f"🚀 Starting upload from: {CSV_FILE}")
start = time.time()

threads = []
for _ in range(NUM_THREADS):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

read_and_queue_csv(CSV_FILE)
queue.join()

# Stop workers
for _ in range(NUM_THREADS):
    queue.put(None)
for t in threads:
    t.join()

save_rejected_rows()
print(f"🎉 All batches uploaded in {time.time() - start:.2f} seconds.")


🚀 Starting upload from: Datasets/skills_data.csv
✅ Batch 1 inserted in 1.53s (100 records)
✅ Batch 1 inserted in 1.52s (100 records)
✅ Batch 1 inserted in 1.73s (100 records)
✅ Batch 1 inserted in 1.74s (100 records)
✅ Batch 2 inserted in 0.36s (100 records)
✅ Batch 2 inserted in 0.31s (100 records)
✅ Batch 2 inserted in 0.42s (100 records)
✅ Batch 2 inserted in 0.26s (63 records)
🎉 All batches uploaded in 2.13 seconds.


In [ ]:
#inserts career description data
import os
import csv
import time
import threading
from datetime import datetime, timezone
import boto3
from queue import Queue

# CONFIGURATION
CSV_FILE = "Datasets/description.csv"
REGION = "us-west-2"
BATCH_SIZE = 100
NUM_THREADS = 4
REJECTED_FILE = "rejected_career_description.csv"

# AWS setup
dynamodb = boto3.resource("dynamodb", region_name=REGION)
table = dynamodb.Table(CAREER_DESCRIPTION_TABLE)

queue = Queue()
rejected_rows = []
lock = threading.Lock()

def current_timestamp():
    return datetime.now(timezone.utc).isoformat()

def process_batch(batch, batch_number):
    start = time.time()
    try:
        with table.batch_writer(overwrite_by_pkeys=['occCode']) as writer:
            for item in batch:
                try:
                    writer.put_item(Item=item)
                except Exception as e:
                    with lock:
                        rejected_rows.append({**item, 'error': str(e)})
        print(f"✅ Batch {batch_number} inserted in {time.time() - start:.2f}s ({len(batch)} records)")
    except Exception as e:
        with lock:
            for item in batch:
                rejected_rows.append({**item, 'error': str(e)})
        print(f"❌ Batch {batch_number} failed with error: {str(e)}")

def worker():
    batch_number = 1
    while True:
        batch = queue.get()
        if batch is None:
            break
        process_batch(batch, batch_number)
        batch_number += 1
        queue.task_done()

def read_and_queue_csv(filepath):
    with open(filepath, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        batch = []
        for row_num, row in enumerate(reader, 2):
            try:
                now = current_timestamp()
                item = {
                    'occCode': str(row['Code']),
                    'description': str(row['Description']),
                    'createdAt': now,
                    'updatedAt': now,
                }
                batch.append(item)
                if len(batch) == BATCH_SIZE:
                    queue.put(batch)
                    batch = []
                if row_num % 1000 == 0:
                    print(f"   ...Processed {row_num} rows so far")
            except Exception as e:
                with lock:
                    rejected_rows.append({**row, 'error': str(e)})
        if batch:
            queue.put(batch)

def save_rejected_rows():
    if rejected_rows:
        keys = rejected_rows[0].keys()
        with open(REJECTED_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(rejected_rows)
        print(f"⚠️ {len(rejected_rows)} records rejected. Saved to {REJECTED_FILE}")

# Main
print(f"🚀 Starting upload from: {CSV_FILE}")
start = time.time()

threads = []
for _ in range(NUM_THREADS):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

read_and_queue_csv(CSV_FILE)
queue.join()

# Stop workers
for _ in range(NUM_THREADS):
    queue.put(None)
for t in threads:
    t.join()

save_rejected_rows()
print(f"🎉 All batches uploaded in {time.time() - start:.2f} seconds.")



🚀 Starting upload from: Datasets/description.csv
   ...Processed 1000 rows so far
✅ Batch 1 inserted in 1.29s (100 records)
✅ Batch 1 inserted in 1.32s (100 records)
✅ Batch 1 inserted in 1.33s (100 records)
✅ Batch 1 inserted in 1.36s (100 records)
✅ Batch 2 inserted in 0.11s (100 records)
✅ Batch 2 inserted in 0.13s (100 records)
✅ Batch 2 inserted in 0.16s (100 records)
✅ Batch 3 inserted in 0.04s (16 records)
✅ Batch 2 inserted in 0.18s (100 records)
✅ Batch 3 inserted in 0.14s (100 records)
✅ Batch 3 inserted in 0.14s (100 records)
🎉 All batches uploaded in 1.64 seconds.


In [ ]:
#insert soc codes data
import os
import csv
import time
import threading
from datetime import datetime, timezone
import boto3
from queue import Queue

# CONFIGURATION
CSV_FILE = "Datasets/unique_occ_codes.csv"
REGION = "us-west-2"
BATCH_SIZE = 100
NUM_THREADS = 4
REJECTED_FILE = "rejected_soc_codes.csv"

# AWS setup
dynamodb = boto3.resource("dynamodb", region_name=REGION)
table = dynamodb.Table(SOC_CODES_TABLE)

queue = Queue()
rejected_rows = []
lock = threading.Lock()

def current_timestamp():
    return datetime.now(timezone.utc).isoformat()

def process_batch(batch, batch_number):
    start = time.time()
    try:
        with table.batch_writer(overwrite_by_pkeys=['occCode']) as writer:
            for item in batch:
                try:
                    writer.put_item(Item=item)
                except Exception as e:
                    with lock:
                        rejected_rows.append({**item, 'error': str(e)})
        print(f"✅ Batch {batch_number} inserted in {time.time() - start:.2f}s ({len(batch)} records)")
    except Exception as e:
        with lock:
            for item in batch:
                rejected_rows.append({**item, 'error': str(e)})
        print(f"❌ Batch {batch_number} failed with error: {str(e)}")

def worker():
    batch_number = 1
    while True:
        batch = queue.get()
        if batch is None:
            break
        process_batch(batch, batch_number)
        batch_number += 1
        queue.task_done()

def read_and_queue_csv(filepath):
    with open(filepath, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        batch = []
        for row_num, row in enumerate(reader, 2):
            try:
                now = current_timestamp()
                item = {
                    'occCode': str(row['OCC_CODE']),
                    'occTitle': str(row['OCC_TITLE']),
                    'createdAt': now,
                    'updatedAt': now,
                }
                batch.append(item)
                if len(batch) == BATCH_SIZE:
                    queue.put(batch)
                    batch = []
                if row_num % 1000 == 0:
                    print(f"   ...Processed {row_num} rows so far")
            except Exception as e:
                with lock:
                    rejected_rows.append({**row, 'error': str(e)})
        if batch:
            queue.put(batch)

def save_rejected_rows():
    if rejected_rows:
        keys = rejected_rows[0].keys()
        with open(REJECTED_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(rejected_rows)
        print(f"⚠️ {len(rejected_rows)} records rejected. Saved to {REJECTED_FILE}")

# Main
print(f"🚀 Starting upload from: {CSV_FILE}")
start = time.time()

threads = []
for _ in range(NUM_THREADS):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

read_and_queue_csv(CSV_FILE)
queue.join()

# Stop workers
for _ in range(NUM_THREADS):
    queue.put(None)
for t in threads:
    t.join()

save_rejected_rows()
print(f"🎉 All batches uploaded in {time.time() - start:.2f} seconds.")


🚀 Starting upload from: Datasets/unique_occ_codes.csv
   ...Processed 1000 rows so far
✅ Batch 1 inserted in 1.06s (100 records)
✅ Batch 1 inserted in 1.07s (100 records)
✅ Batch 1 inserted in 1.10s (100 records)
✅ Batch 1 inserted in 1.09s (100 records)
✅ Batch 2 inserted in 0.25s (100 records)
✅ Batch 2 inserted in 0.24s (100 records)
✅ Batch 2 inserted in 0.27s (100 records)
✅ Batch 2 inserted in 0.30s (100 records)
✅ Batch 3 inserted in 0.07s (43 records)
✅ Batch 3 inserted in 0.18s (100 records)
✅ Batch 3 inserted in 0.19s (100 records)
🎉 All batches uploaded in 1.56 seconds.
